# Feature Selection : Wrapper Methods

Los metodos mas comunes son:

1. Forward Selection
2. Backward elimination
3. Bi-directional elimination (stepwise)

In [2]:
import pandas as pd
import matplotlib as mpl #para graficar por python
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [7]:
boston = pd.read_csv('C:\\Users\\VICTUS\\Documents\\2024\\DATA SCIENCE\\14. Validacion de un Modelo\\boston_house_prices.csv')

In [ ]:
#no correr
print(boston.data.shape)         # dataset dimension
print(boston.feature_names)      # nombre feature
print(boston.target)             # target variable
print(boston.DESCR)              # data description

In [ ]:
import pandas as pd
bos = pd.DataFrame(boston.data, columns = boston.feature_names)
bos['Price'] = boston.target

#DIVIDO ENTRE LA VARIABLE QUE QUIERO PREDECIR Y EL RESTO DE LAS VARIABLES
X = bos.drop("Price", 1)       # feature matrix
y = bos['Price']               # target feature VARIABLE OBJETIVO
bos.head()

In [11]:
import statsmodels.api as sm
def forward_selection(data, target, significance_level=0.05):  #Nombre del dataset - variable target (y) -  nivel de significancia (0.05 generalmente)
    initial_features = data.columns.tolist() #nos traemos todas las columnas del df en una lista
    best_features = [] #hacemos una lista vacia

    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features)) #todas las columnas iniciales - las mejores (la primera vez esta vacia pero va a ir comparando una con otra)
        new_pval = pd.Series(index=remaining_features) #me armo un panda series
        
        for new_column in remaining_features: #a cada columna que se va guardando hacele un modelo de regresion lineal, le va agregando columnas de a poco y va viendo la importancia de la variable
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level): #si el p value es menor al nivel de significancia va agregando las variables importantes
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

In [ ]:
forward_selection(X,y) #aca me tira las variables importantes

# Implementacion de future selection usando MLXTEND

In [13]:
!pip install mlxtend

In [14]:
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib

In [16]:
#Librerias
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

# Sequential Forward Selection(sfs)
sfs = SFS(LinearRegression(),
          k_features=11, #le paso la cantidad de features que yo me quiero quedar, puedo ir variandolas
          forward=True,
          floating=False,
          scoring = 'r2', #scoring
          cv = 0)

In [ ]:
sfs.fit(X, y)
sfs.k_feature_names_     #Lista final de features

#aca me tira cuales son las variables importantes para el modelo

# Backward Selection

- Va a ir eliminando variables que no sean importantes y se queda con el minimo indispensable de variables
- El codigo es parecido al anterior pero no le indico la cantidad de variables que se tiene que quedar

In [18]:
def backward_elimination(data, target,significance_level = 0.05): #df, target, nivel de significancia
    features = data.columns.tolist() #toma todas las features como importantes y las va eliminando
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max() #me quedo con el que de mejor nivel de significancia

        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature) #elimina de la lista original
        else:
            break
    return features

In [ ]:
backward_elimination(X,y)
# aca me tira cuales son finalemente las variables importantes del datset

# Eliminacion Bidireccional (stepwise)

- Es una mezcla de los dos anteriores

In [19]:
def stepwise_selection(data, target,SL_in=0.05,SL_out = 0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break
        else:
            break
    return best_features

In [ ]:
stepwise_selection(X,y) #aca me tira las variables mas importantes